### Without Face Detection

In [ ]:
import cv2 
import time
import os
import hand_tracking as htm 
import numpy as np 
import screen_brightness_control as sbc 
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume 
import pyautogui 
from tkinter import *
from tkinter import messagebox
import tkinter
import face_recognition


# Defining Variables 
global wCam, hCam
wCam, hCam = 480, 450 # Camera Resolution
cap = cv2.VideoCapture(0)
detector = htm.hand_detect(detectionCon=0.65, maxHands=1)

successs = None
global status
global stat
stat = 0
status = []
# cv2.useOptimized()


            
# Function for Screenshot Naming
def screenshot_name():
    timee = time.ctime()
    t = timee.split(" ")[:3]
    inter_t = timee.split(" ")
    inter = inter_t[3].split(':')
    st = ''
    for i in inter:
        st += i
    strr = ''
    for i in t:
        strr += i
    return str(strr + st) # Return current time and day



# Function for Brightness
def brightness():
    print('Brightness Module Sucessfully Loaded')
    br_bar = 0
    br_per = 0
    colorvol = 0
    tipIds = [4, 8, 12, 16, 20]
    pTime = 0
    fingers = []
    while(1):

        frame, img = cap.read()
        img = detector.find_hands(img, draw=True)
        lmlist, bbox = detector.find_position(img, draw=True)

        if len(lmlist) != 0:


                # find distance b/w index & Thumb
                lenth, img, lineinfo = detector.find_distance(4, 8, img)

                # convert volume
                # Finger Distance Range-> 50-300
                # Volume Range-> -96.0 - 0.125
                br_bar = np.interp(lenth, [8, 200], [400, 150])
                br_per = np.interp(lenth, [8, 200], [0, 100])

                # reduce resolution to make it smoother
                smoothness = 5
                br_level = smoothness * round(br_per / smoothness)

                # check fingers up
                # if pinky-finger is down to set volume
                fingers = detector.fingerup()
                if fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                    sbc.set_brightness(br_per)
                    cv2.circle(
                        img, (lineinfo[4], lineinfo[5]), 10, (0, 0, 255), cv2.FILLED)
                    colorvol = (0, 0, 255)
                    time.sleep(.1)
                else:
                    colorvol = (255, 0, 0)

                if np.sum(fingers) == 5:
                    cv2.destroyAllWindows()
                    global stat
                    slide()
                    return

        # Drawings
        cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 2)
        cv2.rectangle(img, (50, int(br_bar)), (85, 400),
                      (255, 0, 0), cv2.FILLED)  # volume-bar
        cv2.putText(img, f'{str(int(br_per))}%', (40, 450),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1)  # volume percentage

        volset = int(sbc.get_brightness())
        cv2.putText(img, f'Brightness Level:{str(int(volset))}', (280, 40),
                    cv2.FONT_HERSHEY_COMPLEX, 1, colorvol, 2)  # set brightness number

        # Frame Rate
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, f'FPS:{str(int(fps))} ', (30, 60),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow("brightness", img)
        cv2.waitKey(1)


        
# Function for Volume Control
def volume():
    print('Volume Sucess')
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volrange = volume.GetVolumeRange()
    minvol = volrange[0]
    maxvol = volrange[1]
    vol = 0
    volbar = 300
    volper = 0
    bbox = []
    area = 0
    ptime = 0
    smoothness = 0
    colorvol = (255, 0, 0)
    tipIds = [4, 8, 12, 16, 20]
    fingers= []

    while(1):
        frame, img = cap.read()
        img = detector.find_hands(img, draw=True)
        lmlist, bbox = detector.find_position(img, draw=True)

        if len(lmlist) != 0:

            # filter based on size
            area = (bbox[2]-bbox[0]) * (bbox[3]-bbox[1])//100


            # find distance b/w index & Thumb
            lenth, img, lineinfo = detector.find_distance(4, 8, img)

            # convert volume
            # Finger Distance Range-> 50-300
            # Volume Range-> -96.0 - 0.125
            volbar = np.interp(lenth, [8, 200], [400, 150])
            volper = np.interp(lenth, [8, 200], [0, 100])

            # reduce resolution to make it smoother
            smoothness = 2
            volper = smoothness * round(volper / smoothness)

            # check fingers up
            # if pinky-finger is down to set volume
            fingers = detector.fingerup()
                     
            if fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                volume.SetMasterVolumeLevelScalar(volper/100, None)
                cv2.circle(
                    img, (lineinfo[4], lineinfo[5]), 10, (0, 0, 255), cv2.FILLED)
                colorvol = (0, 0, 255)
            else:
                colorvol = (255, 0, 0)

            if np.sum(fingers) == 5: 
                cv2.destroyAllWindows()
                global stat
                slide()
                return

            # Drawings
            cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 2)
            cv2.rectangle(img, (50, int(volbar)), (85, 400),
                          (255, 0, 0), cv2.FILLED)  # volume-bar
            cv2.putText(img, f'{str(int(volper))}%', (40, 450),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1)  # volume percentage
            volset = int(volume.GetMasterVolumeLevelScalar() * 100)
            cv2.putText(img, f'Vol Set:{str(int(volset))}', (450, 40),
                        cv2.FONT_HERSHEY_COMPLEX, 1, colorvol, 2)  # set volume

            # Frame Rate
            ctime = time.time()
            fps = 1 / ((ctime-ptime))  # calculate FPS
            ptime = ctime
            cv2.putText(img, f'FPS:{str(int(fps))} ', (30, 60),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

            cv2.imshow('volume', img)
            cv2.waitKey(1)

            
            
# Minimize Maximize Function
def min_max():
    print('minimize maximize function called')
    br_bar= 0
    br_per= 0
    colorvol= 0
    tipIds = [4, 8, 12, 16, 20]
    pTime= 0
    lenth= 0
    fingers = []
    while(1):

        frame, img= cap.read()
        img= detector.find_hands(img, draw=True)
        lmlist, bbox= detector.find_position(img, draw=True)

        if len(lmlist) != 0:

                #filter based on size
                area= (bbox[2]-bbox[0]) * (bbox[3]-bbox[1])//100
                fingers= detector.fingerup()
                lenth, img, lineinfo= detector.find_distance(4,8,img)
 
                    
                if lenth < 100 and fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                    pyautogui.getActiveWindow().minimize()
                if lenth > 130 and lenth < 250 and fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                    pyautogui.getActiveWindow().maximize()

                if np.sum(fingers) == 5:
                    cv2.destroyAllWindows()
                    global stat
                    slide()
                    return

                # Frame Rate
                cTime = time.time()
                fps = 1 / (cTime - pTime)
                pTime = cTime

                cv2.putText(img, f'FPS:{str(int(fps))} ', (30,60), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)

                cv2.imshow("Image", img)
                cv2.waitKey(1)
            

            
# Function for Display size control (Zoom-in & Zoom-out)
def zoom():
    print('Zoom function called')
    br_bar = 0
    br_per = 0
    colorvol = 0
    tipIds = [4, 8, 12, 16, 20]
    pTime = 0
    lenth = 0
    fingers= []
    while(1):

        frame, img = cap.read()
        img = detector.find_hands(img, draw=True)
        lmlist, bbox = detector.find_position(img, draw=True)

        if len(lmlist) != 0:

            # filter based on size
            area = (bbox[2]-bbox[0]) * (bbox[3]-bbox[1])//100
            fingers = detector.fingerup()

            if lenth < 100 and fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                pyautogui.hotkey('ctrl', '+')
                time.sleep(1)

            if np.sum(fingers) == 5: 
                pyautogui.hotkey('ctrl', '0')
                cv2.destroyAllWindows()
                global stat
                slide()
                return

            # Frame Rate
            cTime = time.time()
            fps = 1 / (cTime - pTime)
            pTime = cTime

            cv2.putText(img, f'FPS:{str(int(fps))} ', (30, 60),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

            cv2.imshow("zoom", img)
            cv2.waitKey(1)


# ----------------------------------------------------------------------------------------------

            
# Function for Mode Detection
def slide():
    tipIds = [4, 8, 12, 16, 20]
#     global cap

    # Read Camera
#     cap = cv2.VideoCapture(0)
    cap.set(3, wCam)
    cap.set(4, hCam)
    while(1):
        success, img = cap.read()
        img = detector.find_hands(img)
        lmlist, bbox = detector.find_position(img, draw=True)
        fingers = []

        # Detect Fingers
        if len(lmlist) != 0:
            fingers = detector.fingerup()
        
        # 1.) Play-Pause (Last three finger gesture)
        if np.sum(fingers) == 3 and fingers[0] == 0 and fingers[1] == 0 and fingers[2] == 1 and fingers[3] == 1 and fingers[4] == 1:
            print('play pause module')
            pyautogui.press("playpause")
            time.sleep(1) # Time delay

        # 2.) shut down (pinky finger)
        if np.sum(fingers) == 1 and fingers[4] == 1 and fingers[0] == 0 and fingers[1] == 0  and fingers[2] ==0  and fingers[3] == 0:
            print('shut down gesture captured')
            ws = tkinter.Tk()
            
            res= tkinter.messagebox.askyesno('prompt' , 'Want to leave?')
            if res == True:
                os.system("shutdown /s /t 1")
            
            elif res == False:
                pass
            else:
                tkinter.messagebox.showerror('error', 'somthing went wrong!')
            
            ws.destroy()
            ws.mainloop()
            

        # 3.) Brightness (Mode: Index Finger )
        if np.sum(fingers) == 1 and fingers[1] == 1 and fingers[0] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 0:
            status.append(1)
            print("Brightness Module")
#             cv2.destroyAllWindows()
            brightness()
            return

        
        # 4.) Zoom (Mode: Three fingers)
        if np.sum(fingers) == 3 and fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 1 and fingers[4] == 0 and fingers[0] == 0:
            status.append(3)
            print('zoom module')
#             cv2.destroyAllWindows()
            zoom()
            return

        
        # 5.) Volume ( Mode: V - sign gesture)
        if np.sum(fingers) == 2 and fingers[1] == 1 and fingers[2] == 1 and fingers[0] == 0 and fingers[3] == 0 and fingers[4] == 0:
            status.append(2)
#             cv2.destroyAllWindows()
            volume()
            return
        
        
        # 6.) Minimize - Maximize (Mode: 4-fingers gesture)
        if np.sum(fingers) == 4 and fingers[0] == 0 and fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 1 and fingers[4] == 1:
            status.append(4)
#             cv2.destroyAllWindows()
            min_max()
            return
        
        
        # 7.) Undo ( thumb and pinky finger gesture )
        if np.sum(fingers) == 2 and fingers[4] == 1 and fingers[0] == 1 and fingers[1] == 0 and fingers[2] == 0 and fingers[3] == 0:
            print('undo gesture captured')
            pyautogui.keyDown('ctrl') 
            pyautogui.press('z')  
            pyautogui.keyUp('ctrl') 
            time.sleep(2)
        
        
        # 8.) Screenshot (index, pinky and thumb finger gesture)
        if np.sum(fingers) == 3 and fingers[4] == 1 and fingers[1] == 1 and fingers[0] == 1 and fingers[2] == 0 and fingers[3] == 0:
            lll = ''
            scr = pyautogui.screenshot()

            user_name = str(os.getcwd().split('\\')[2])
            # user_name 
            pathh = (f"/Users/{user_name}/Desktop/Gesture_SCR")
            # os.mkdir(pathh)
            if not os.path.exists(pathh):
                os.mkdir(pathh)

            lll = screenshot_name()
            scr.save(f"{pathh}/{lll}.jpg")
            time.sleep(3)

            
        # 9.) Save (thumb gesture)
        if np.sum(fingers) == 1 and fingers[0] == 1 and fingers[1] == 0 and fingers[3] == 0 and fingers[4] == 0 and fingers[2] == 0:
            pyautogui.keyDown('ctrl')
            pyautogui.press('s')
            pyautogui.keyUp('ctrl')
            time.sleep(2)

        cv2.putText(img, '1. Brightness', (20, 50),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
        cv2.putText(img, '2. Volume', (20, 100),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('img', img)
        
            
        # 10.) (ALT + F4) ( index + thumb gesture )
        if np.sum(fingers) == 2 and fingers[0] ==1 and fingers[1] == 1 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 0:
            pyautogui.getActiveWindowTitle()
            time.sleep(0.5)
            pyautogui.keyDown('alt')
            pyautogui.press('F4')
            pyautogui.keyUp('alt')
        
        # 11.) Restart ( Index + pinky finger gesture )
        if fingers[0] ==0 and fingers[1] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 0:
            print('Restart gesture captured')
            ws = tkinter.Tk()
            
            res= tkinter.messagebox.askyesno('prompt' , 'Want to leave?')
            if res == True:
                os.system("shutdown /r /t 1")
            
            elif res == False:
                pass
            else:
                tkinter.messagebox.showerror('error', 'somthing went wrong!')
            
            ws.destroy()
            ws.mainloop()

        if cv2.waitKey(1) & 0xFF == ord("e"):
            cap.release()
            cv2.destroyAllWindows()
# -------------------------------------------------------------------------------------------------

# Face Recongnition


path= 'Attendees/'
images= []
img_names= []

mylist= os.listdir(path)
mylist= mylist[1:]

for classes in mylist:
    img= cv2.imread(f'{path}/{classes}')
    images.append(img)
    img_names.append(os.path.splitext(classes)[0])

# print(img_names)

encodlist= []
def find_encodings(images):
#     encodlist= []
    for img in images:
        img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encods= face_recognition.face_encodings(img)[0]
        encodlist.append(encods)
    
    return encodlist
find_encodings(images)

# cap= cv2.VideoCapture(0)
# while True:
#         frame, im= cap.read()
#         img= cv2.resize(im, (0,0), None, 0.25, 0.25)
#         img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
#         curfaces_in_frame= face_recognition.face_locations(img)
#         encod_curframe= face_recognition.face_encodings(img, curfaces_in_frame) #finding encodings of img & current frame
        
#         for encodface, face_loc in zip(encod_curframe,curfaces_in_frame):
#             match= face_recognition.compare_faces(encodlist, encodface)
#             facedis= face_recognition.face_distance(encodlist, encodface)
# #             print(facedis) # prints face distance
#             match_index= np.argmin(facedis)
#             face_name= img_names[match_index]
        
#             if match[match_index]:
#                 print(f"Approved, {face_name}'s Face Detected")
#                 stat = slide()
# #                 cap.release()
#             else:
#                 print('Accses Denied')
# #                 cap.release()
                
                

# Release the camera and all resources

                
# --------------------------------------------------------------------------------------------------

            
stat = slide()
print(stat)
if stat == 1:
    status = []
    stat = 0
    brightness()
if stat == 1 and status[0] == 2:
    status = []
    stat = 0
    volume()
if stat == 1 and status[0] == 3:
    status = []
    stat = 0
    zoom()
if stat == 1 and status[0] == 4:
    status= []
    stat = 0
    min_max()

## Final Prototype (With Face Detection)

In [2]:
import cv2 
import time
import os
import hand_tracking as htm 
import numpy as np 
import screen_brightness_control as sbc 
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume 
import pyautogui 
from tkinter import *
from tkinter import messagebox
import tkinter
import face_recognition


# Defining Variables 
global wCam, hCam
wCam, hCam = 480, 450 # Camera Resolution
cap = cv2.VideoCapture(0)
detector = htm.hand_detect(detectionCon=0.65, maxHands=1)

successs = None
global status
global stat
stat = 0
status = []
cv2.useOptimized()


            
# Function for Screenshot Naming
def screenshot_name():
    timee = time.ctime()
    t = timee.split(" ")[:3]
    inter_t = timee.split(" ")
    inter = inter_t[3].split(':')
    st = ''
    for i in inter:
        st += i
    strr = ''
    for i in t:
        strr += i
    return str(strr + st) # Return current time and day



# Function for Brightness
def brightness():
    print('Brightness Module Sucessfully Loaded')
    br_bar = 0
    br_per = 0
    colorvol = 0
    tipIds = [4, 8, 12, 16, 20]
    pTime = 0
    fingers = []
    while(1):

        frame, img = cap.read()
        img = detector.find_hands(img, draw=True)
        lmlist, bbox = detector.find_position(img, draw=True)

        if len(lmlist) != 0:


                # find distance b/w index & Thumb
                lenth, img, lineinfo = detector.find_distance(4, 8, img)

                # convert volume
                # Finger Distance Range-> 50-300
                # Volume Range-> -96.0 - 0.125
                br_bar = np.interp(lenth, [8, 200], [400, 150])
                br_per = np.interp(lenth, [8, 200], [0, 100])

                # reduce resolution to make it smoother
                smoothness = 5
                br_level = smoothness * round(br_per / smoothness)

                # check fingers up
                # if pinky-finger is down to set volume
                fingers = detector.fingerup()
                if fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                    sbc.set_brightness(br_per)
                    cv2.circle(
                        img, (lineinfo[4], lineinfo[5]), 10, (0, 0, 255), cv2.FILLED)
                    colorvol = (0, 0, 255)
                    time.sleep(.1)
                else:
                    colorvol = (255, 0, 0)

                if np.sum(fingers) == 5:
                    cv2.destroyAllWindows()
                    global stat
                    slide()
                    return

        # Drawings
        cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 2)
        cv2.rectangle(img, (50, int(br_bar)), (85, 400),
                      (255, 0, 0), cv2.FILLED)  # volume-bar
        cv2.putText(img, f'{str(int(br_per))}%', (40, 450),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1)  # volume percentage

        volset = int(sbc.get_brightness())
        cv2.putText(img, f'Brightness Level:{str(int(volset))}', (280, 40),
                    cv2.FONT_HERSHEY_COMPLEX, 1, colorvol, 2)  # set brightness number

        # Frame Rate
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, f'FPS:{str(int(fps))} ', (30, 60),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow("brightness", img)
        cv2.waitKey(1)


        
# Function for Volume Control
def volume():
    print('Volume Sucess')
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volrange = volume.GetVolumeRange()
    minvol = volrange[0]
    maxvol = volrange[1]
    vol = 0
    volbar = 300
    volper = 0
    bbox = []
    area = 0
    ptime = 0
    smoothness = 0
    colorvol = (255, 0, 0)
    tipIds = [4, 8, 12, 16, 20]
    fingers= []

    while(1):
        frame, img = cap.read()
        img = detector.find_hands(img, draw=True)
        lmlist, bbox = detector.find_position(img, draw=True)

        if len(lmlist) != 0:

            # filter based on size
            area = (bbox[2]-bbox[0]) * (bbox[3]-bbox[1])//100


            # find distance b/w index & Thumb
            lenth, img, lineinfo = detector.find_distance(4, 8, img)

            # convert volume
            # Finger Distance Range-> 50-300
            # Volume Range-> -96.0 - 0.125
            volbar = np.interp(lenth, [8, 200], [400, 150])
            volper = np.interp(lenth, [8, 200], [0, 100])

            # reduce resolution to make it smoother
            smoothness = 2
            volper = smoothness * round(volper / smoothness)

            # check fingers up
            # if pinky-finger is down to set volume
            fingers = detector.fingerup()
                     
            if fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                volume.SetMasterVolumeLevelScalar(volper/100, None)
                cv2.circle(
                    img, (lineinfo[4], lineinfo[5]), 10, (0, 0, 255), cv2.FILLED)
                colorvol = (0, 0, 255)
            else:
                colorvol = (255, 0, 0)

            if np.sum(fingers) == 5: 
                cv2.destroyAllWindows()
                global stat
                slide()
                return

            # Drawings
            cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 2)
            cv2.rectangle(img, (50, int(volbar)), (85, 400),
                          (255, 0, 0), cv2.FILLED)  # volume-bar
            cv2.putText(img, f'{str(int(volper))}%', (40, 450),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1)  # volume percentage
            volset = int(volume.GetMasterVolumeLevelScalar() * 100)
            cv2.putText(img, f'Vol Set:{str(int(volset))}', (450, 40),
                        cv2.FONT_HERSHEY_COMPLEX, 1, colorvol, 2)  # set volume

            # Frame Rate
            ctime = time.time()
            fps = 1 / ((ctime-ptime))  # calculate FPS
            ptime = ctime
            cv2.putText(img, f'FPS:{str(int(fps))} ', (30, 60),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

            cv2.imshow('volume', img)
            cv2.waitKey(1)

            
            
# Minimize Maximize Function
def min_max():
    print('minimize maximize function called')
    br_bar= 0
    br_per= 0
    colorvol= 0
    tipIds = [4, 8, 12, 16, 20]
    pTime= 0
    lenth= 0
    fingers = []
    while(1):

        frame, img= cap.read()
        img= detector.find_hands(img, draw=True)
        lmlist, bbox= detector.find_position(img, draw=True)

        if len(lmlist) != 0:

                #filter based on size
                area= (bbox[2]-bbox[0]) * (bbox[3]-bbox[1])//100
                fingers= detector.fingerup()
                lenth, img, lineinfo= detector.find_distance(4,8,img)
 
                    
                if lenth < 100 and fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                    pyautogui.getActiveWindow().minimize()
                if lenth > 130 and lenth < 250 and fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                    pyautogui.getActiveWindow().maximize()

                if np.sum(fingers) == 5:
                    cv2.destroyAllWindows()
                    global stat
                    slide()
                    return

                # Frame Rate
                cTime = time.time()
                fps = 1 / (cTime - pTime)
                pTime = cTime

                cv2.putText(img, f'FPS:{str(int(fps))} ', (30,60), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)

                cv2.imshow("Image", img)
                cv2.waitKey(1)
            

            
# Function for Display size control (Zoom-in & Zoom-out)
def zoom():
    print('Zoom function called')
    br_bar = 0
    br_per = 0
    colorvol = 0
    tipIds = [4, 8, 12, 16, 20]
    pTime = 0
    lenth = 0
    fingers= []
    while(1):

        frame, img = cap.read()
        img = detector.find_hands(img, draw=True)
        lmlist, bbox = detector.find_position(img, draw=True)

        if len(lmlist) != 0:

            # filter based on size
            area = (bbox[2]-bbox[0]) * (bbox[3]-bbox[1])//100
            fingers = detector.fingerup()

            if lenth < 100 and fingers[4] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[0] == 1 and fingers[1] == 1:
                pyautogui.hotkey('ctrl', '+')
                time.sleep(1)

            if np.sum(fingers) == 5: 
                pyautogui.hotkey('ctrl', '0')
                cv2.destroyAllWindows()
                global stat
                slide()
                return

            # Frame Rate
            cTime = time.time()
            fps = 1 / (cTime - pTime)
            pTime = cTime

            cv2.putText(img, f'FPS:{str(int(fps))} ', (30, 60),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

            cv2.imshow("zoom", img)
            cv2.waitKey(1)


# ----------------------------------------------------------------------------------------------

            
# Function for Mode Detection
def slide():
    tipIds = [4, 8, 12, 16, 20]


    # Read Camera
#     cap = cv2.VideoCapture(0)
    cap.set(3, wCam)
    cap.set(4, hCam)
    while(1):
        success, img = cap.read()
        img = detector.find_hands(img)
        lmlist, bbox = detector.find_position(img, draw=True)
        fingers = []

        # Detect Fingers
        if len(lmlist) != 0:
            fingers = detector.fingerup()
        
        # 1.) Play-Pause (Last three finger gesture)
        if np.sum(fingers) == 3 and fingers[0] == 0 and fingers[1] == 0 and fingers[2] == 1 and fingers[3] == 1 and fingers[4] == 1:
            print('play pause module')
            pyautogui.press("playpause")
            time.sleep(1) # Time delay

        # 2.) shut down (pinky finger)
        if np.sum(fingers) == 1 and fingers[4] == 1 and fingers[0] == 0 and fingers[1] == 0  and fingers[2] ==0  and fingers[3] == 0:
            print('shut down gesture captured')
            ws = tkinter.Tk()
            
            res= tkinter.messagebox.askyesno('prompt' , 'Want to leave?')
            if res == True:
                os.system("shutdown /s /t 1")
            
            elif res == False:
                pass
            else:
                tkinter.messagebox.showerror('error', 'somthing went wrong!')
            
            ws.destroy()
            ws.mainloop()
            

        # 3.) Brightness (Mode: Index Finger )
        if np.sum(fingers) == 1 and fingers[1] == 1 and fingers[0] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 0:
            status.append(1)
            print("Brightness Module")
#             cv2.destroyAllWindows()
            brightness()
            return

        
        # 4.) Zoom (Mode: Three fingers)
        if np.sum(fingers) == 3 and fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 1 and fingers[4] == 0 and fingers[0] == 0:
            status.append(3)
            print('zoom module')
#             cv2.destroyAllWindows()
            zoom()
            return

        
        # 5.) Volume ( Mode: V - sign gesture)
        if np.sum(fingers) == 2 and fingers[1] == 1 and fingers[2] == 1 and fingers[0] == 0 and fingers[3] == 0 and fingers[4] == 0:
            status.append(2)
#             cv2.destroyAllWindows()
            volume()
            return
        
        
        # 6.) Minimize - Maximize (Mode: 4-fingers gesture)
        if np.sum(fingers) == 4 and fingers[0] == 0 and fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 1 and fingers[4] == 1:
            status.append(4)
#             cv2.destroyAllWindows()
            min_max()
            return
        
        
        # 7.) Undo ( thumb and pinky finger gesture )
        if np.sum(fingers) == 2 and fingers[4] == 1 and fingers[0] == 1 and fingers[1] == 0 and fingers[2] == 0 and fingers[3] == 0:
            print('undo gesture captured')
            pyautogui.keyDown('ctrl') 
            pyautogui.press('z')  
            pyautogui.keyUp('ctrl') 
            time.sleep(2)
        
        
        # 8.) Screenshot (index, pinky and thumb finger gesture)
        if np.sum(fingers) == 3 and fingers[4] == 1 and fingers[1] == 1 and fingers[0] == 1 and fingers[2] == 0 and fingers[3] == 0:
            lll = ''
            scr = pyautogui.screenshot()

            user_name = str(os.getcwd().split('\\')[2])
            # user_name 
            pathh = (f"/Users/{user_name}/Desktop/Gesture_SCR")
            # os.mkdir(pathh)
            if not os.path.exists(pathh):
                os.mkdir(pathh)

            lll = screenshot_name()
            scr.save(f"{pathh}/{lll}.jpg")
            time.sleep(3)

            
        # 9.) Save (thumb gesture)
        if np.sum(fingers) == 1 and fingers[0] == 1 and fingers[1] == 0 and fingers[3] == 0 and fingers[4] == 0 and fingers[2] == 0:
            pyautogui.keyDown('ctrl')
            pyautogui.press('s')
            pyautogui.keyUp('ctrl')
            time.sleep(2)
        
            
        # 10.) (ALT + F4) ( index + thumb gesture )
        if np.sum(fingers) == 2 and fingers[0] ==1 and fingers[1] == 1 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 0:
            pyautogui.getActiveWindowTitle()
            time.sleep(0.5)
            pyautogui.keyDown('alt')
            pyautogui.press('F4')
            pyautogui.keyUp('alt')
        
        # 11.) Restart ( Index + pinky finger gesture )
        if fingers[0] ==0 and fingers[1] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 0:
            print('Restart gesture captured')
            ws = tkinter.Tk()
            
            res= tkinter.messagebox.askyesno('prompt' , 'Want to leave?')
            if res == True:
                os.system("shutdown /r /t 1")

            elif res == False:
                pass
            else:
                tkinter.messagebox.showerror('error', 'somthing went wrong!')
            
            ws.destroy()
            ws.mainloop()

            
        cv2.putText(img, '1. Brightness (Index Finger)', (17, 30),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '2. Volume (V-Sign)', (17, 55),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '3. Screenshot (index + pinky + thumb)', (17, 75),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '4. shut-down (pinky finger)', (17, 95),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '5. Save (Thumb)', (17, 115),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1) 
        cv2.putText(img, '6. Minimize - Maximize (4-fingers)', (17, 135),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '7. Restart (Index + pinky)', (17, 155),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '8. Undo (thumb + pinky)', (17, 175),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '9. Zoom (First Three fingers)', (17, 195),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '10. Play-Pause (Last three finger )', (17, 215),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(img, '11. (ALT + F4) (index + thumb)', (17, 235),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
        
        cv2.imshow('img', img)    
            
            
        if cv2.waitKey(1) & 0xFF == ord("e"):
            cap.release()
            cv2.destroyAllWindows()
            
# -------------------------------------------------------------------------------------------------

# Face Recongnition


path= 'Attendees/'
images= []
img_names= []

mylist= os.listdir(path)
mylist= mylist[1:]
# print(mylist)


for classes in mylist:
    img= cv2.imread(f'{path}/{classes}')
    images.append(img)
    img_names.append(os.path.splitext(classes)[0])
# print(img_names)

encodlist= []
def find_encodings(images):
#     encodlist= []
    for img in images:
        img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encods= face_recognition.face_encodings(img)[0]
        encodlist.append(encods)
    
    return encodlist
find_encodings(images)

def face_matching():
    cap= cv2.VideoCapture(0)
    while True:
            frame, im= cap.read()
            img= cv2.resize(im, (0,0), None, 0.25, 0.25)
            img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            curfaces_in_frame= face_recognition.face_locations(img)
            encod_curframe= face_recognition.face_encodings(img, curfaces_in_frame) #finding encodings of img & current frame

            for encodface, face_loc in zip(encod_curframe,curfaces_in_frame):
                match= face_recognition.compare_faces(encodlist, encodface)
                facedis= face_recognition.face_distance(encodlist, encodface)
    #             print(facedis) # prints face distance
                match_index= np.argmin(facedis)
                face_name= img_names[match_index]

                if match[match_index]:
                    print(f"Approved, {face_name}'s Face Detected")

                    return True
                    cap.release()
                else:
                    print('Accses Denied')
                    cap.release()
                    return False
                
                

# Release the camera and all resources

                
# --------------------------------------------------------------------------------------------------

            
if face_matching():
    stat = slide()
print(stat)
if stat == 1:
    status = []
    stat = 0
    brightness()
if stat == 1 and status[0] == 2:
    status = []
    stat = 0
    volume()
if stat == 1 and status[0] == 3:
    status = []
    
    stat = 0
    zoom()
if stat == 1 and status[0] == 4:
    status= []
    stat = 0
    min_max()

TypeError: create_int(): incompatible function arguments. The following argument types are supported:
    1. (arg0: int) -> mediapipe.python._framework_bindings.packet.Packet

Invoked with: 0.65

In [ ]:
pip uninstall mediapipe